In [1]:
# Input the type of analysis to be run (options are 'line_darkframe', 'line_lightframe', 'spot_darkframe') 
type_of_analysis = 'line_lightframe'

# Input the color channel to be analyze (options are 'red', 'green', 'blue' and 'gray')
color_channel = 'red'

In [2]:
import os
import multiprocessing as mp
from multiprocessing.pool import ThreadPool as Pool
import pandas as pd

def run_batchfile(each_file):
    # Execute a batch file using the OS system command
    os.system(each_file)
    return 1


def get_each_df(each_dir):
    # Read a CSV file from a directory and add a 'group' column
    each_df = pd.read_csv(os.path.join(each_dir, 'image.csv'))
    each_df['group'] = os.path.basename(each_dir)
    return each_df

# this may stay this way or may move to yaml argparse 
para_texts = {
     'line_darkframe': "{\"row0\":608, \"row1\":978, \"col0\":450, \"col1\":1700, \"median_blur_size\":7, \"sobel_size\":11, \"dark_frame\":1, \"gblur_size\":30, \"crop_right_extra\":1, \"row_offset\":20, \"col_offset\":260, \"border\":[500, 800], \"color_channel\":[\"gray\", \"gray\"], \"signal_type\":[\"line\", \"line\"], \"do_detrend\":1, \"ndetrend\":5, \"nsignal\":3, \"nblur\":400, \"top_fraction\":0.05, \"contour_mode\":1, \"contour_method\":2, \"rect_height\":10, \"edge_gap\":40}",
     'line_lightframe':  "{\"row0\":608, \"row1\":900, \"col0\":680, \"col1\":1500, \"median_blur_size\":7, \"sobel_size\":5, \"dark_frame\":0, \"gblur_size\":30, \"crop_right_extra\":1, \"row_offset\":40, \"col_offset\":30, \"border\":[190, 465], \"color_channel\":[\"red\", \"red\"], \"signal_type\":[\"line\", \"line\"], \"do_detrend\":1, \"ndetrend\":5, \"nsignal\":3, \"nblur\":400, \"top_fraction\":0.05, \"contour_mode\":1, \"contour_method\":2, \"rect_height\":10, \"edge_gap\":40}",
     'spot_darkframe': "{\"row0\":644, \"row1\":924, \"col0\":840, \"col1\":1380, \"median_blur_size\":7, \"sobel_size\":5, \"dark_frame\":1, \"gblur_size\":10, \"crop_right_extra\":0, \"row_offset\":40, \"col_offset\":30, \"border\":[300], \"color_channel\":[\"gray\"], \"signal_type\":[\"spot\"], \"do_detrend\":1, \"ndetrend\":5, \"nsignal\":3, \"nblur\":400, \"top_fraction\":0.05, \"contour_mode\":1, \"contour_method\":2, \"rect_height\":10, \"edge_gap\":40}",
}

In [3]:
if __name__ == '__main__':
    # Get the corresponding parameter text
    para_text = para_texts[type_of_analysis]

    # Find all subdirectories in the current directory
    dir = [each[0] for each in os.walk('.')]
    dir = dir[1:]  # Eliminate the first one, which is the current directory '.'
    dir = [each for each in dir if ('.idea' not in each and '__pycache__' not in each 
                                    and '.git' not in each and '.ipynb_checkpoints' not in each)]

    # Create batch files for each subdirectory    
    file = [os.path.basename(each) + '.bat' for each in dir]
    for each_dir, each_file in zip(dir, file):
        # Prepare the command to run the analysis script with the directory and parameters
        out_string = 'python "Hamilton_image_analysis.py" ' + '"' + each_dir + '" "' + para_text.replace('"','\\"') + '"\n'
        
        # Write the command to the batch file
        out_file = open(each_file, 'w')
        out_file.write(out_string)
        out_file.close()

    # Run the batch files in parallel using multiprocessing
    pool = Pool(mp.cpu_count())
    pool.map(run_batchfile, file)
    pool.close()
    pool.join()

    # Compile all data from the processed directories into a single DataFrame
    df = pd.concat([get_each_df(each_dir) for each_dir in dir])
    
    # Extract the 'guid' from the filename and add it as a new column
    df['guid'] = df['filename'].str.split('_', n=1, expand=True).iloc[:,0].astype(int)
    
    # Save the compiled data to a CSV file
    df.to_csv('all_results.csv', index=False)